In [ ]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [ ]:
import gensim
import pandas as pd

df = pd.read_json('/content/Cell_Phones_and_Accessories_5.json', lines=True)
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"
...,...,...,...,...,...,...,...,...,...
194434,A1YMNTFLNDYQ1F,B00LORXVUE,eyeused2loveher,"[0, 0]",Works great just like my original one. I reall...,5,This works just perfect!,1405900800,"07 21, 2014"
194435,A15TX8B2L8B20S,B00LORXVUE,Jon Davidson,"[0, 0]",Great product. Great packaging. High quality a...,5,Great replacement cable. Apple certified,1405900800,"07 21, 2014"
194436,A3JI7QRZO1QG8X,B00LORXVUE,Joyce M. Davidson,"[0, 0]","This is a great cable, just as good as the mor...",5,Real quality,1405900800,"07 21, 2014"
194437,A1NHB2VC68YQNM,B00LORXVUE,Nurse Farrugia,"[0, 0]",I really like it becasue it works well with my...,5,I really like it becasue it works well with my...,1405814400,"07 20, 2014"


In [ ]:
df.shape

(194439, 9)

In [ ]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [ ]:
model.build_vocab(review_text, progress_per=1000)

In [ ]:
model.train(review_text, total_examples=model.corpus_count, epochs=50)

(0, 2986414200)

In [ ]:
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")

In [ ]:
model.wv.most_similar("good")
vocab_size = len(model.wv)

[('decent', 0.8160923719406128),
 ('great', 0.720307469367981),
 ('fantastic', 0.6882827877998352),
 ('nice', 0.6528382301330566),
 ('bad', 0.6257654428482056),
 ('reasonable', 0.6216623783111572),
 ('terrific', 0.6128300428390503),
 ('cool', 0.6011943221092224),
 ('exceptional', 0.5975220203399658),
 ('outstanding', 0.5974794030189514)]

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time


df = pd.read_json('/content/Cell_Phones_and_Accessories_5.json', lines=True)

review_text = df.reviewText
ratings = df.overall
review_text,ratings



(0         They look good and stick good! I just don't li...
 1         These stickers work like the review says they ...
 2         These are awesome and make my phone look so st...
 3         Item arrived in great time and was in perfect ...
 4         awesome! stays on, and looks great. can be use...
                                 ...                        
 194434    Works great just like my original one. I reall...
 194435    Great product. Great packaging. High quality a...
 194436    This is a great cable, just as good as the mor...
 194437    I really like it becasue it works well with my...
 194438    product as described, I have wasted a lot of m...
 Name: reviewText, Length: 194439, dtype: object,
 0         4
 1         5
 2         5
 3         4
 4         5
          ..
 194434    5
 194435    5
 194436    5
 194437    5
 194438    5
 Name: overall, Length: 194439, dtype: int64)

In [ ]:
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

# https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate/34294022
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

review_text = review_text.map(remove_URL) # map(lambda x: remove_URL(x))
review_text = review_text.map(remove_punct)
review_text

0         They look good and stick good I just dont like...
1         These stickers work like the review says they ...
2         These are awesome and make my phone look so st...
3         Item arrived in great time and was in perfect ...
4         awesome stays on and looks great can be used o...
                                ...                        
194434    Works great just like my original one I really...
194435    Great product Great packaging High quality and...
194436    This is a great cable just as good as the more...
194437    I really like it becasue it works well with my...
194438    product as described I have wasted a lot of mo...
Name: reviewText, Length: 194439, dtype: object

In [ ]:
# remove stopwords
# pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine
# has been programmed to ignore, both when indexing entries for searching and when retrieving them
# as the result of a search query.
stop = set(stopwords.words("english"))

# https://stackoverflow.com/questions/5486337/how-to-remove-stop-words-using-nltk-or-python
def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

review_text = review_text.map(remove_stopwords)
review_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0         look good stick good dont like rounded shape a...
1         stickers work like review says stick great sta...
2         awesome make phone look stylish used one far a...
3         item arrived great time perfect condition howe...
4         awesome stays looks great used multiple apple ...
                                ...                        
194434    works great like original one really need extr...
194435    great product great packaging high quality app...
194436    great cable good expensive apple one husband d...
194437    really like becasue works well life proof case...
194438    product described wasted lot money cords would...
Name: reviewText, Length: 194439, dtype: object

In [ ]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(review_text)
counter

Counter({'look': 15503,
         'good': 58328,
         'stick': 3338,
         'dont': 34353,
         'like': 72694,
         'rounded': 582,
         'shape': 1651,
         'always': 10187,
         'bumping': 98,
         'siri': 531,
         'kept': 2516,
         'popping': 320,
         'irritating': 204,
         'wont': 8717,
         'buy': 19069,
         'product': 38026,
         'stickers': 651,
         'work': 28148,
         'review': 11661,
         'says': 3435,
         'great': 66609,
         'stay': 4358,
         'phone': 173629,
         'super': 5108,
         'stylish': 1820,
         'share': 743,
         'sister': 752,
         'awesome': 5693,
         'make': 16788,
         'used': 25217,
         'one': 85971,
         'far': 13160,
         'almost': 7993,
         'year': 4945,
         'believe': 2820,
         'quality': 27945,
         'item': 9495,
         'arrived': 4405,
         'time': 35813,
         'perfect': 15349,
         'condition

In [ ]:
num_unique_words = len(counter)

review_text.shape[0]

train_size = int(review_text.shape[0] * 0.8)

train_df = review_text[:train_size]
val_df = review_text[train_size:]

# split text and labels
train_sentences = train_df.to_numpy()
train_labels = ratings[:train_size].to_numpy()
val_sentences = val_df.to_numpy()
val_labels = ratings[train_size:].to_numpy()

train_sentences.shape, val_sentences.shape

((155551,), (38888,))

In [ ]:
# Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence of integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) # fit only to training

word_index = tokenizer.word_index
word_index

{'phone': 1,
 'case': 2,
 'one': 3,
 'like': 4,
 'great': 5,
 'screen': 6,
 'use': 7,
 'good': 8,
 'would': 9,
 'battery': 10,
 'well': 11,
 'iphone': 12,
 'get': 13,
 'really': 14,
 'product': 15,
 'charge': 16,
 'time': 17,
 'dont': 18,
 'also': 19,
 'works': 20,
 'charger': 21,
 'much': 22,
 'love': 23,
 'price': 24,
 'little': 25,
 'back': 26,
 'work': 27,
 'fit': 28,
 'nice': 29,
 'quality': 30,
 'even': 31,
 'easy': 32,
 'im': 33,
 'bought': 34,
 'still': 35,
 'protector': 36,
 'got': 37,
 'used': 38,
 'ive': 39,
 'using': 40,
 'better': 41,
 'doesnt': 42,
 '2': 43,
 'recommend': 44,
 'device': 45,
 'two': 46,
 'fits': 47,
 'usb': 48,
 'buy': 49,
 'charging': 50,
 'first': 51,
 'need': 52,
 'cover': 53,
 'put': 54,
 'looks': 55,
 'new': 56,
 'without': 57,
 '5': 58,
 'cases': 59,
 'power': 60,
 'protection': 61,
 'headset': 62,
 'way': 63,
 'hard': 64,
 'car': 65,
 'around': 66,
 'could': 67,
 'cable': 68,
 'phones': 69,
 'thing': 70,
 'make': 71,
 'want': 72,
 'didnt': 73,
 'go'

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)
print(train_sentences[10:15])
print(train_sequences[10:15])

['case fits perfectly s4 keeps powerd day cant complain recommend'
 'first battery case galaxy s4 s4 fits well slim doesnt add much weight galaxy s4 doubles battery life charge either battery phone handy onoff switch leds indicate level chargethe battery case came time packaged well well worth price'
 'performs exactly advertised sturdily builtand provides lots boost exactly supposed easy insert phone definitely 5 star experience dont know would without case love love love'
 'unlike rechargeable battery cases powerbear lasts 2 whole days doesnt heat like ones completely fascinated ultra light sleek design case using mophie case couldnt wear often like hot brick pocket hence always leave homeon contrary powerbear never take cant even tell difference also build super strong manner even though dropped phone times shock resistant technology wont let single thing happen case phone powerbear case became extension phone never take charge night charges phone case battery life two days also sho

In [ ]:
# Pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Max number of words in a sequence
max_length = 40

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, val_padded.shape

((155551, 40), (38888, 40))

In [ ]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

case fits perfectly s4 keeps powerd day cant complain recommend
[2, 47, 110, 471, 458, 36973, 103, 91, 916, 44]
[    2    47   110   471   458 36973   103    91   916    44     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]


In [ ]:
# Check reversing the indices

# flip (key, value)
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [ ]:
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

decoded_text = decode(train_sequences[10])

print(train_sequences[10])
print(decoded_text)

[2, 47, 110, 471, 458, 36973, 103, 91, 916, 44]
case fits perfectly s4 keeps powerd day cant complain recommend


In [ ]:
from tensorflow import keras
from keras import layers


# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

# Word embeddings give us a way to use an efficient, dense representation in which similar words have
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a
# dense vector of floating point values (the length of the vector is a parameter you specify).

model_nlp = keras.models.Sequential()
model_nlp.add(layers.Embedding(num_unique_words, 32, input_length=40))

# The layer will take as input an integer matrix of size (batch, input_length),
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.


model_nlp.add(layers.LSTM(64))
model_nlp.add(layers.Dense(6, activation="softmax"))

model_nlp.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 32)            6985184   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 6)                 390       
                                                                 
Total params: 7,010,406
Trainable params: 7,010,406
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.01)
metrics = ["accuracy"]

model_nlp.compile(loss=loss, optimizer=optim, metrics=metrics)
model_nlp.fit(train_padded, train_labels, epochs=10, validation_data=(val_padded, val_labels), verbose=2)

Epoch 1/20
4861/4861 - 555s - loss: 1.0177 - accuracy: 0.6001 - val_loss: 0.8725 - val_accuracy: 0.6580 - 555s/epoch - 114ms/step
Epoch 2/20
4861/4861 - 567s - loss: 0.8944 - accuracy: 0.6443 - val_loss: 0.8840 - val_accuracy: 0.6508 - 567s/epoch - 117ms/step
Epoch 3/20
4861/4861 - 537s - loss: 0.8132 - accuracy: 0.6792 - val_loss: 0.9033 - val_accuracy: 0.6416 - 537s/epoch - 110ms/step
Epoch 4/20
4861/4861 - 539s - loss: 0.7215 - accuracy: 0.7201 - val_loss: 0.9678 - val_accuracy: 0.6371 - 539s/epoch - 111ms/step
Epoch 5/20
4861/4861 - 528s - loss: 0.6292 - accuracy: 0.7596 - val_loss: 1.0421 - val_accuracy: 0.6180 - 528s/epoch - 109ms/step
Epoch 6/20
4861/4861 - 529s - loss: 0.5438 - accuracy: 0.7936 - val_loss: 1.2067 - val_accuracy: 0.6218 - 529s/epoch - 109ms/step
Epoch 7/20
4861/4861 - 529s - loss: 0.4708 - accuracy: 0.8215 - val_loss: 1.3481 - val_accuracy: 0.6167 - 529s/epoch - 109ms/step
Epoch 8/20
4861/4861 - 536s - loss: 0.4125 - accuracy: 0.8434 - val_loss: 1.6169 - val_acc

In [ ]:
predictions = model_nlp.predict(train_padded)

max_pred = predictions.argmax(axis=1)
# print(max_pred[10])

print(train_sentences[10:20])
print(train_labels[10:20])
print(max_pred[10:20])

4861/4861 [==============================] - 48s 10ms/step
['case fits perfectly s4 keeps powerd day cant complain recommend'
 'first battery case galaxy s4 s4 fits well slim doesnt add much weight galaxy s4 doubles battery life charge either battery phone handy onoff switch leds indicate level chargethe battery case came time packaged well well worth price'
 'performs exactly advertised sturdily builtand provides lots boost exactly supposed easy insert phone definitely 5 star experience dont know would without case love love love'
 'unlike rechargeable battery cases powerbear lasts 2 whole days doesnt heat like ones completely fascinated ultra light sleek design case using mophie case couldnt wear often like hot brick pocket hence always leave homeon contrary powerbear never take cant even tell difference also build super strong manner even though dropped phone times shock resistant technology wont let single thing happen case phone powerbear case became extension phone never take cha

In [ ]:
model.save('my_model.h5')